In [10]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd

# If you installed scipy, this will work. If not, see fallback below.
try:
    import scipy.stats as stats
    SCIPY_AVAILABLE = True
except Exception:
    SCIPY_AVAILABLE = False

# Make src importable
sys.path.append("../src")

from data_loader import LeadDataLoader


# ----------------------------
# 0) Load data
# ----------------------------
DATA_PATH = "../data/raw/Lead Scoring.csv"
loader = LeadDataLoader(DATA_PATH)
df = loader.load_data()

print("Loaded:", Path(DATA_PATH).name)
print("Shape:", df.shape)
df.head()


# ----------------------------
# 1) Quick ID integrity checks
# ----------------------------
ID_COLS = ["Prospect ID", "Lead Number"]
TARGET_COL = "Converted"

print("\n# ID integrity")
for col in ID_COLS:
    if col in df.columns:
        print(f"{col}: missing={df[col].isna().sum()}, duplicates={df[col].duplicated().sum()}, n_unique={df[col].nunique(dropna=False)}")
    else:
        print(f"{col}: NOT FOUND")


# ----------------------------
# 2) Missing values overview
# ----------------------------
print("\n# Missing values (top 25)")
missing_df = (
    df.isna().sum().to_frame("missing_count")
      .assign(missing_pct=lambda x: x["missing_count"] / len(df) * 100)
      .sort_values("missing_count", ascending=False)
)
display(missing_df.head(25))

# Only columns with missing
display(missing_df[missing_df["missing_count"] > 0])


# ----------------------------
# 3) Detect "pseudo-missing" tokens in object columns
# ----------------------------
PSEUDO_MISSING_TOKENS = [
    "Select", "Not Provided", "None", "", " "
]

obj_cols = df.select_dtypes(include=["object"]).columns.tolist()

def find_pseudo_missing(df, columns, tokens):
    hits = []
    for col in columns:
        s = df[col].astype(str)
        for token in tokens:
            # token == "" is special: any string contains ""
            if token == "":
                # identify truly empty strings
                has = (df[col] == "").any()
            elif token == " ":
                has = (df[col] == " ").any()
            else:
                has = s.str.contains(token, case=False, na=False).any()
            if has:
                hits.append((col, token))
    return pd.DataFrame(hits, columns=["column", "token"]).sort_values(["column", "token"])

pseudo_hits = find_pseudo_missing(df, obj_cols, PSEUDO_MISSING_TOKENS)
print("\n# Pseudo-missing token hits")
display(pseudo_hits)


# ----------------------------
# 4) Helper: show top values per column (useful for leakage clues)
# ----------------------------
def show_top_values(df, col, top_n=15):
    print(f"\n===== {col} =====")
    print("dtype:", df[col].dtype)
    print("n_unique (incl NaN):", df[col].nunique(dropna=False))
    display(df[col].value_counts(dropna=False).head(top_n))

# Example: uncomment to inspect a few columns
# for c in ["Lead Quality", "Tags", "Lead Profile", "Last Activity", "Last Notable Activity"]:
#     if c in df.columns:
#         show_top_values(df, c)


# ----------------------------
# 5) Leakage/Missingness test: is_missing vs Converted
# ----------------------------
def missing_vs_conversion(df, col, target=TARGET_COL):
    tmp = df[[col, target]].copy()
    tmp["is_missing"] = tmp[col].isna()

    agg = tmp.groupby("is_missing")[target].agg(count="count", conversion_rate="mean")
    agg["conversion_rate"] = agg["conversion_rate"].round(4)
    agg["count_pct"] = (agg["count"] / len(df) * 100).round(2)

    # Lift: conv(not missing) - conv(missing)
    if set(agg.index) == {False, True}:
        lift = float(agg.loc[False, "conversion_rate"] - agg.loc[True, "conversion_rate"])
    else:
        lift = np.nan

    print(f"\n==== Missingness vs Conversion: {col} ====")
    display(agg)

    print("Lift (not-missing minus missing):", round(lift, 4))
    return agg, lift

def missing_chi2(df, col, target=TARGET_COL):
    if not SCIPY_AVAILABLE:
        print(f"{col}: scipy not available -> skipping chi2. (Install scipy or use the fallback below.)")
        return None

    tmp = df[[col, target]].copy()
    tmp["is_missing"] = tmp[col].isna()

    contingency = pd.crosstab(tmp["is_missing"], tmp[target])
    # if a row/col is missing (rare), ensure full 2x2
    contingency = contingency.reindex(index=[False, True], columns=[0, 1], fill_value=0)

    chi2, p, dof, expected = stats.chi2_contingency(contingency)
    return {"chi2": chi2, "p_value": p, "dof": dof, "contingency": contingency, "expected": expected}

def run_leakage_missingness_suite(df, cols, target=TARGET_COL):
    rows = []
    for col in cols:
        if col not in df.columns:
            print(f"Skipping (not found): {col}")
            continue

        agg, lift = missing_vs_conversion(df, col, target=target)

        chi2_out = missing_chi2(df, col, target=target)
        if chi2_out is not None:
            p = chi2_out["p_value"]
            rows.append({
                "column": col,
                "missing_count": int(df[col].isna().sum()),
                "missing_pct": float(df[col].isna().mean() * 100),
                "conv_rate_missing": float(agg.loc[True, "conversion_rate"]) if True in agg.index else np.nan,
                "conv_rate_not_missing": float(agg.loc[False, "conversion_rate"]) if False in agg.index else np.nan,
                "lift_not_missing_minus_missing": float(lift),
                "chi2_p_value": float(p),
            })
        else:
            rows.append({
                "column": col,
                "missing_count": int(df[col].isna().sum()),
                "missing_pct": float(df[col].isna().mean() * 100),
                "conv_rate_missing": float(agg.loc[True, "conversion_rate"]) if True in agg.index else np.nan,
                "conv_rate_not_missing": float(agg.loc[False, "conversion_rate"]) if False in agg.index else np.nan,
                "lift_not_missing_minus_missing": float(lift),
                "chi2_p_value": np.nan,
            })

    result = pd.DataFrame(rows).sort_values("missing_pct", ascending=False)
    # format
    result["missing_pct"] = result["missing_pct"].round(2)
    result["lift_not_missing_minus_missing"] = result["lift_not_missing_minus_missing"].round(4)
    if "chi2_p_value" in result.columns:
        result["chi2_p_value"] = result["chi2_p_value"].map(lambda x: np.nan if pd.isna(x) else float(x))
    return result


# ----------------------------
# 6) Choose columns to check for leakage via missingness
# (based on your missingness results)
# ----------------------------
COLS_TO_CHECK = [
    "Lead Quality",
    "Asymmetrique Profile Index",
    "Asymmetrique Activity Score",
    "Asymmetrique Activity Index",
    "Asymmetrique Profile Score",
    "Tags",
    "What matters most to you in choosing a course",
    "Lead Profile",
    "What is your current occupation",
    "Country",
    "How did you hear about X Education",
    "Specialization",
    "City",
    "Page Views Per Visit",
    "TotalVisits",
    "Last Activity",
    "Lead Source",
    "Last Notable Activity",
]

leakage_missingness_report = run_leakage_missingness_suite(df, COLS_TO_CHECK, target=TARGET_COL)
print("\n# Missingness -> potential leakage report")
display(leakage_missingness_report)


# ----------------------------
# 7) Optional: Inspect top values for the most suspicious columns
# (high missing + big lift)
# ----------------------------
# Pick top 5 by absolute lift (if available)
tmp_report = leakage_missingness_report.copy()
tmp_report["abs_lift"] = tmp_report["lift_not_missing_minus_missing"].abs()
top_suspects = tmp_report.sort_values("abs_lift", ascending=False).head(5)["column"].tolist()

print("\n# Top suspects by absolute lift (inspect values):", top_suspects)

for col in top_suspects:
    if col in df.columns:
        show_top_values(df, col, top_n=20)


# ----------------------------
# 8) Fallback if scipy isn't installed (optional)
# This section only runs if SCIPY_AVAILABLE is False.
# It provides a quick, non-chi2 indicator using difference in rates.
# ----------------------------
if not SCIPY_AVAILABLE:
    print("\nNOTE: scipy is not installed. You can install it via:")
    print("  pip install scipy")
    print("\nUntil then, rely on lift + counts to judge missingness association.")

Loaded: Lead Scoring.csv
Shape: (9240, 37)

# ID integrity
Prospect ID: missing=0, duplicates=0, n_unique=9240
Lead Number: missing=0, duplicates=0, n_unique=9240

# Missing values (top 25)


,missing_count,missing_pct
Lead Quality,4767,51.590909
Asymmetrique Profile Index,4218,45.649351
Asymmetrique Activity Score,4218,45.649351
Asymmetrique Activity Index,4218,45.649351
Asymmetrique Profile Score,4218,45.649351
Tags,3353,36.287879
What matters most to you in choosing a course,2709,29.318182
Lead Profile,2709,29.318182
What is your current occupation,2690,29.112554
Country,2461,26.634199


,missing_count,missing_pct
Lead Quality,4767,51.590909
Asymmetrique Profile Index,4218,45.649351
Asymmetrique Activity Score,4218,45.649351
Asymmetrique Activity Index,4218,45.649351
Asymmetrique Profile Score,4218,45.649351
Tags,3353,36.287879
What matters most to you in choosing a course,2709,29.318182
Lead Profile,2709,29.318182
What is your current occupation,2690,29.112554
Country,2461,26.634199



# Pseudo-missing token hits


,column,token
4,City,Select
1,How did you hear about X Education,Select
3,Lead Profile,Select
0,Specialization,Select
2,Tags,Not Provided


...

,count,conversion_rate,count_pct
is_missing,,,
False,9204,0.3837,99.61
True,36,0.8056,0.39


Lift (not-missing minus missing): -0.4219

==== Missingness vs Conversion: Last Notable Activity ====


,count,conversion_rate,count_pct
is_missing,,,
False,9240,0.3854,100.0


Lift (not-missing minus missing): nan


ValueError: The internally computed table of expected frequencies has a zero element at (np.int64(1), np.int64(0)).

Lead Quality                                     4767
Asymmetrique Profile Index                       4218
Asymmetrique Activity Score                      4218
Asymmetrique Activity Index                      4218
Asymmetrique Profile Score                       4218
Tags                                             3353
What matters most to you in choosing a course    2709
Lead Profile                                     2709
What is your current occupation                  2690
Country                                          2461
How did you hear about X Education               2207
Specialization                                   1438
City                                             1420
Page Views Per Visit                              137
TotalVisits                                       137
Last Activity                                     103
Lead Source                                        36
dtype: int64

,missing_count,missing_pct
Lead Quality,4767,51.590909
Asymmetrique Profile Index,4218,45.649351
Asymmetrique Activity Score,4218,45.649351
Asymmetrique Activity Index,4218,45.649351
Asymmetrique Profile Score,4218,45.649351
Tags,3353,36.287879
What matters most to you in choosing a course,2709,29.318182
Lead Profile,2709,29.318182
What is your current occupation,2690,29.112554
Country,2461,26.634199



===== Prospect ID =====
n_unique: 9240
Prospect ID
571b5c8e-a5b2-4d57-8574-f2ffb06fdeff    1
7927b2df-8bba-4d29-b9a2-b6e0beafe620    1
2a272436-5132-4136-86fa-dcc88c88f482    1
8cc8c611-a219-4f35-ad23-fdfd2656bd8a    1
0cc2df48-7cf4-4e39-9de9-19797f9b38cc    1
3256f628-e534-4826-9d63-4a8b88782852    1
2058ef08-2858-443e-a01f-a9237db2f5ce    1
9fae7df4-169d-489b-afe4-0f3d752542ed    1
20ef72a2-fb3b-45e0-924e-551c5fa59095    1
cfa0128c-a0da-4656-9d47-0aa4e67bf690    1
Name: count, dtype: int64

===== Lead Number =====
n_unique: 9240
Lead Number
579533    1
660737    1
660728    1
660727    1
660719    1
660681    1
660680    1
660673    1
660664    1
660624    1
Name: count, dtype: int64

===== Lead Origin =====
n_unique: 5
Lead Origin
Landing Page Submission    4886
API                        3580
Lead Add Form               718
Lead Import                  55
Quick Add Form                1
Name: count, dtype: int64

===== Lead Source =====
n_unique: 22
Lead Source
Google             


===== Prospect ID =====
n_unique: 9240
Prospect ID
571b5c8e-a5b2-4d57-8574-f2ffb06fdeff    1
7927b2df-8bba-4d29-b9a2-b6e0beafe620    1
2a272436-5132-4136-86fa-dcc88c88f482    1
8cc8c611-a219-4f35-ad23-fdfd2656bd8a    1
0cc2df48-7cf4-4e39-9de9-19797f9b38cc    1
3256f628-e534-4826-9d63-4a8b88782852    1
2058ef08-2858-443e-a01f-a9237db2f5ce    1
9fae7df4-169d-489b-afe4-0f3d752542ed    1
20ef72a2-fb3b-45e0-924e-551c5fa59095    1
cfa0128c-a0da-4656-9d47-0aa4e67bf690    1
Name: count, dtype: int64

===== Lead Origin =====
n_unique: 5
Lead Origin
Landing Page Submission    4886
API                        3580
Lead Add Form               718
Lead Import                  55
Quick Add Form                1
Name: count, dtype: int64

===== Lead Source =====
n_unique: 22
Lead Source
Google              2868
Direct Traffic      2543
Olark Chat          1755
Organic Search      1154
Reference            534
Welingak Website     142
Referral Sites       125
Facebook              55
NaN              

Prospect ID contains ''
Lead Origin contains ''
Lead Source contains ''
Do Not Email contains ''
Do Not Call contains ''
Last Activity contains ''
Country contains ''
Specialization contains 'Select'
Specialization contains ''
How did you hear about X Education contains 'Select'
How did you hear about X Education contains ''
What is your current occupation contains ''
What matters most to you in choosing a course contains ''
Search contains ''
Magazine contains ''
Newspaper Article contains ''
X Education Forums contains ''
Newspaper contains ''
Digital Advertisement contains ''
Through Recommendations contains ''
Receive More Updates About Our Courses contains ''
Tags contains 'Not Provided'
Tags contains ''
Lead Quality contains ''
Update me on Supply Chain Content contains ''
Get updates on DM Content contains ''
Lead Profile contains 'Select'
Lead Profile contains ''
City contains 'Select'
City contains ''
Asymmetrique Activity Index contains ''
Asymmetrique Profile Index contains 

Missing Prospect ID: 0
Missing Lead Number: 0
Duplicate Prospect ID: 0
Duplicate Lead Number: 0
